In [2]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
from datetime import datetime
from loguru import logger
import numpy as np
import spatialdata as sd
from napari_spatialdata import Interactive

from multiplex_pipeline.utils.config_loaders import load_analysis_settings
from multiplex_pipeline.object_quantification.controller import QuantificationController

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load analysis settings

In [ ]:
# load analysis configuration
settings_path = r'C:\sdata\analysis_settings_DT1-HW.yaml'

settings = load_analysis_settings(settings_path)
settings

AnalysisConfig(general=GeneralSettings(image_dir='R:/CellDive/DT1-HW/DT1-HW_Final', analysis_name='DT1-HW_Analysis', local_analysis_dir='C:/sdata', remote_analysis_dir='', log_dir=None), core_detection=CoreDetectionSettings(detection_image='DT1-HW_1.0.4_R000_DAPI__FINAL_F.ome.tif', core_info_file_path=None, im_level=6, min_area=2000, max_area=10000, min_iou=0.8, min_st=0.9, min_int=15, frame=4), core_cutting=CoreCuttingSettings(cores_dir_tif=None, cores_dir_output=None, include_channels=None, exclude_channels=None, use_markers=None, ignore_markers=['Antibody1'], margin=0, mask_value=0, transfer_cleanup_enabled=True, core_cleanup_enabled=True), additional_elements=[NormalizeStep(category='image_transformer', type='normalize', input='DAPI', output='DAPI_norm', keep=False, parameters=Params(low=1.0, high=99.8)), NormalizeStep(category='image_transformer', type='normalize', input='ECad', output='ECad_norm', keep=False, parameters=Params(low=1.0, high=99.8)), InstansegStep(category='object_

### Define the logger

In [4]:
log_file = settings.log_dir_path / f"quantification_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
l = logger.add(lambda msg: print(msg, end=""))
l = logger.add(log_file, level="DEBUG", enqueue=True)

### Define cores for the analysis

In [5]:
core_dir = settings.analysis_dir / 'cores'
path_list = [core_dir / f for f in os.listdir(core_dir)]
path_list.sort()
path_list

[WindowsPath('C:/sdata/DT1-HW_Analysis/cores/Core_000.zarr'),
 WindowsPath('C:/sdata/DT1-HW_Analysis/cores/Core_001.zarr'),
 WindowsPath('C:/sdata/DT1-HW_Analysis/cores/Core_002.zarr'),
 WindowsPath('C:/sdata/DT1-HW_Analysis/cores/Core_003.zarr')]

### Setup

In [6]:
# setup quantification controllers
quant_controller_list = [] 
qc_prefix = settings.qc.prefix
for quant in settings.quant:

    table_name = quant.name
    masks_keys = quant.masks
    connect_to_mask = quant.layer_connection

    logger.info(f"Setting up quantification controller for '{table_name}' table with masks {masks_keys} and connection to '{connect_to_mask}' mask")

    controller = QuantificationController(
        table_name=table_name,
        mask_keys=masks_keys,
        connect_to_mask=connect_to_mask,
        overwrite=True,
        quantify_qc = True,
        qc_prefix = qc_prefix,
    )

    quant_controller_list.append(controller) 

2025-11-06 14:12:30.728 | INFO     | __main__:<module>:10 - Setting up quantification controller for 'instanseg_table' table with masks {'nucleus': 'instanseg_nucleus', 'cell': 'instanseg_cell', 'ring': 'ring', 'cyto': 'cytoplasm'} and connection to 'instanseg_cell' mask


### Quantify

In [7]:
for sd_path in path_list:
    
    # load data
    logger.info(f'Processing {sd_path.name}')
    sdata = sd.read_zarr(sd_path)

    # run quantification
    for controller in quant_controller_list:
        controller.run(sdata)

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-11-06 14:12:35.299 | INFO     | __main__:<module>:4 - Processing Core_000.zarr


version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-11-06 14:12:39.761 | INFO     | multiplex_pipeline.object_quantification.controller:validate_sdata_as_input:229 - Channels not specified. Quantifying all existing channels (12).
2025-11-06 14:12:39.762 | WARNING  | multiplex_pipeline.object_quantification.controller:prepare_to_overwrite:195 - Table 'instanseg_table' already exists and will be overwritten.
2025-11-06 14:12:44.505 | INFO     | multiplex_pipeline.object_quantification.controller:prepare_to_overwrite:200 - Existing table 'instanseg_table' deleted from sdata.
2025-11-06 14:13:51.728 | INFO     | multiplex_pipeline.object_quantification.controller:run:253 - Prepared masks for quantification.
2025-11-06 14:13:51.728 | INFO     | multiplex_pipeline.object_quantification.controller:build_obs:73 - Quantifying morphology features for mask 'nucleus'
2025-11-06 14:14:10.892 | INFO     | multiplex_pipeline.object_quantification.controller:build_obs:73 - Quantifying morphology features for mask 'cell'
2025-11-06 14:14:33.781 | I

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\spatialdata\models\models.py:1144: UserWarning: Converting `region_key: region` to categorical dtype.
  return convert_region_column_to_categorical(adata)


...

2025-11-06 14:41:32.122 | SUCCESS  | multiplex_pipeline.object_quantification.controller:run:338 - Quantification complete. Table 'instanseg_table' written to C:\sdata\DT1-HW_Analysis\cores\Core_002.zarr
2025-11-06 14:41:32.129 | INFO     | __main__:<module>:4 - Processing Core_003.zarr


version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-11-06 14:41:35.274 | INFO     | multiplex_pipeline.object_quantification.controller:validate_sdata_as_input:224 - Quantifying 12 user-specified channels: ['AP2A', 'AP2B', 'AR', 'CD45', 'CK14', 'CK818', 'DAPI', 'ECad', 'ER', 'FOXA1', 'GATA3', 'HER2'].
2025-11-06 14:41:35.275 | WARNING  | multiplex_pipeline.object_quantification.controller:prepare_to_overwrite:195 - Table 'instanseg_table' already exists and will be overwritten.
2025-11-06 14:41:36.408 | INFO     | multiplex_pipeline.object_quantification.controller:prepare_to_overwrite:200 - Existing table 'instanseg_table' deleted from sdata.
2025-11-06 14:41:42.428 | INFO     | multiplex_pipeline.object_quantification.controller:run:253 - Prepared masks for quantification.
2025-11-06 14:41:42.429 | INFO     | multiplex_pipeline.object_quantification.controller:build_obs:73 - Quantifying morphology features for mask 'nucleus'
2025-11-06 14:41:53.276 | INFO     | multiplex_pipeline.object_quantification.controller:build_obs:73 - Qu

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\spatialdata\models\models.py:1144: UserWarning: Converting `region_key: region` to categorical dtype.
  return convert_region_column_to_categorical(adata)


2025-11-06 14:43:46.479 | INFO     | multiplex_pipeline.object_quantification.controller:run:295 - Quantification complete. Resulting AnnData has 9876 observations and 97 variables.
2025-11-06 14:43:46.523 | INFO     | multiplex_pipeline.object_quantification.qc_shape_masker:validate_sdata:34 - Table instanseg_table present in the spatialdata object.
2025-11-06 14:43:46.524 | INFO     | multiplex_pipeline.object_quantification.qc_shape_masker:validate_sdata:43 - Centroids: centroid_cell present in the anndata table instanseg_table.
2025-11-06 14:43:47.647 | SUCCESS  | multiplex_pipeline.object_quantification.controller:run:338 - Quantification complete. Table 'instanseg_table' written to C:\sdata\DT1-HW_Analysis\cores\Core_003.zarr


### Sneak peek

In [10]:
sd_path = path_list[1]
sdata = sd.read_zarr(sd_path)

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requeste

In [11]:
from napari_spatialdata import Interactive

Interactive(sdata)

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\napari\utils\_proxies.py:103: FutureWarning: The `_dock_widgets` property is private and should not be used in any plugin code. Please use the `dock_widgets` property instead.
  return super().__getattr__(name)


2025-11-06 14:54:47.748 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-06 14:54:48.355 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.


c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


2025-11-06 14:55:17.303 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-06 14:55:17.312 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-06 14:55:17.313 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-06 14:55:53.974 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-06 14:55:53.977 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.


c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


2025-11-06 14:56:19.234 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-06 14:56:19.243 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-11-06 14:56:19.244 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
